# Analogue to Digital conversion
In this notebook, the signal analysis techniques developed throughout the course are applied to an analogue to digital converter.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import scipy as sp
from scipy.signal import *
import scipy.linalg as spl
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters

The following parameters can be specified.  

Parameter | Meaning
--------- | -------
<code>b</code> |The number of converter bits (e.g. 3)
<code>Ts</code> |Sampling interval (inverse of sampling rate, e.g. 1/8000)
<code>f</code> |Frequency of sinewave (e.g. 200 Hz)
<code>R</code> |Range of ADC is +/- R (e.g. 0.5 V)
<code>A</code> |Signal peak to peak amplitude (should be < R)
<code>O</code> |Oversampling rate (e.g. 4)
<code>D</code> |Decimation filter length (odd number) (e.g. 311)
<code>plot_samples</code> |Number of sample periods to plot (e.g. 50)
<code>duration</code> |Length of time to compute in unts of Ts/plot_samples (e.g. 65536)
<code>points_per_sample</code> |For plotting purposes (e.g. 16, must be >= O)

In [ ]:
b = 3
Ts = 1 / 8000
f = 200
R = 0.5
A = R * 0.85
O = 4
D = 311
plot_samples = 50
duration = 65536
points_per_sample = 16

### Function definitions
We are going to use the minimum variance spectral estimation technique, so we will require a function to compute the autocorrelation values for M lags, and also a function to compute the MVSE estimate.

In [ ]:
def Autocorrelation(x, M):
    """
        Calculate autocorrelation of x with maximum lag M.
        
        INPUT:
            x - vector to be correlated
            M - maximum correlation lag
        
        RETURN:
            acf - autocorrelation of x with lag M
        
    """

    #### Step 1 - initialise the index to identify the data block
    #### We don't need to copy the data block as python can select
    #### the block at the call time of the FFT.  i is reserved for
    #### sqrt(-1), so call it index instead
    index = 0
    
    #### Step 2 - compute the FFT of x_i(n)
    X = sp.fft.fft(x[index*(M+1):(index+1)*(M+1)], 2*M+2)

    #### Steps 3 to 6 are repeated, accumulating their results in
    #### a vector.  We need to initialise the vector first

    result = np.zeros(2*M+2)

    # It is also helpful to generate the vector of [ 1 -1 1 ... ]

    phase_shift = np.power((-1), np.arange(0, 2*M+2))

    # Do the repetition until we run out of data
    while (True):

        #### Step 3 - compute X_i(k)X^*_i(k) and store

        result = result + np.multiply(np.conj(X), X).real

        #### Step 4 - increment i

        index = index + 1

        # Check to see if we have used all of the data
        if (index*(M+1) > len(x)):
            break

        #### Step 5 - compute the transform for the next block

        if ((index+1)*(M+1) <= len(x)):
            nextX = sp.fft.fft(x[index*(M+1):(index+1)*(M+1)], 2*M+2)
        else:
            # We don't always have a full block of data at the end
            # of the record, but we still need to process it
            nextX = sp.fft.fft(x[index*(M+1):], 2*M+2)

        #### Step 6 - add in the product of the previous and next
        ####          transforms, with the phase shift

        result = result + np.multiply(np.multiply(phase_shift, np.conj(X)), nextX)

        #### Step 7 - repeat steps 3 to 6 until all of the data
        #### has been used.  Before we do this, we need
        #### to make X = nextX

        X = nextX

    #### Step 8 - inverse FFT

    time_domain = sp.fft.ifft(result, 2*M+2)

    #### Step 9 = present only the first M+1 values

    acf = np.divide(time_domain[0:M+1], len(x)).real
    
    return acf

In [ ]:
def final_spectrum(downsampled, FS):
    """
       Compute the final spectrum using minimum variance spectral estimation
    """
    
    # Compute the autocorrelation, and create the Toeplitz matrix
    
    # Normalise to signal power assuming sinewave input
    rxx = Autocorrelation(x = downsampled/(R*np.sqrt(2)), M = p)
    Rxx = spl.toeplitz(rxx)
    
    # Perform the eigenvalue decomposition
    [d,v] = np.linalg.eig(Rxx)
    
    # Invert the elements of the diagonal matrix, and store as a vector <br>
    # eps avoids a divide by zero
    U = np.divide(np.ones(p+1), (abs(d)+np.finfo(float).eps)) 
    
    # Transform the eigenvectors.  The result is a matrix of dimensions
    # fs x p - each eigenvector is transformed
    V = abs(np.fft.fft(v.T,FS))**2
    
    # Then compute the final spectrum by combining the transformed variables
    # and normalising by the length of the correlation vector
    Px = 10*np.log10(p) - 10*np.log10(np.dot(V.T,U))
    
    return Px

An analogue to digital converter requires a sample and hold device, and also a quantizer.

In [ ]:
def hold_quantize(divider = 1, *, samples):
    """
        Implement the hold and quantize the signals
    """
    
    # Implement the hold
    sh_output = np.convolve(samples, np.ones(int(points_per_sample/divider)))

    # Quantize the signal
    # round an array to integer
    quantized = np.round((2**b-1) * sh_output / (2*R+np.finfo(float).eps))/(2**b-1)
    
    return(sh_output, quantized)

The notebook will be displaying sampled signals, as well as spectral plots.  The following two functions are used for this purpose.

In [ ]:
def plot_amplitude(s, sh_output, quantized, name):
    """
       Plot the amplitude
    """
    
    plt.figure(figsize = (12, 6))
    plt.rcParams.update({'font.size': 16})
    
    plt.plot(tim, s, label = 'Input')
    plt.plot(tim, sh_output[0:duration+1],label = 'Sample and Hold')
    plt.plot(tim, quantized[0:duration+1], label = 'Quantized')
    
    plt.xlim([0, plot_samples*Ts])
    plt.xlabel('Time (s)')
    plt.ylim([-R*2, R*2])
    plt.ylabel('Amplitude')
    plt.legend(prop={'size': 15})
    
    if not is_jupyter(): plt.savefig(name)

In [ ]:
def plot_magnitude(x, y, name):
    """
       Plot the magnitude of normalised frequency
    """
        
    plt.figure(figsize = (12, 6))
    plt.rcParams.update({'font.size': 16})
    
    plt.plot(x, y)
    
    plt.xlabel('Normalised frequency')
    plt.xlim([0, 0.5])
    plt.ylabel('Magnitude (dB)')
    plt.ylim([-55, 15])
    
    if not is_jupyter(): plt.savefig(name)

### The spectral analysis parameters
Define the analysis parameters, and set up the axes for plotting

In [ ]:
p = 256
fs = 2048
ofs = fs * O
frequency = np.arange(0, fs) / fs
os_frequency = np.arange(0, ofs) / ofs

add_bits = np.ceil((np.log2(O)-1)/2)
ns_add_bits = np.ceil(5*np.log2(O)/2)

# Analysis sampling rate for plotting continuous time curves
Ta = Ts / points_per_sample
# Set up time axis index
t = np.arange(0, duration+1)
# Set up the plotting axis
tim = t * Ta
# Identify the sampling points as a vector
sampling_points = (np.mod(t, points_per_sample) == 0)
oversampling_points = (np.mod(t, points_per_sample/O) == 0)
# and index them
downsample_index = np.arange(0, duration/points_per_sample+1, dtype=int) * points_per_sample 
os_downsample_index = (np.arange(0, duration*O/points_per_sample+1) * points_per_sample / O).astype(int)

# Define the input signal
s = A * np.sin(t*Ta*2*np.pi*f)

## The Analogue to Digital Converter

### Simple conversion
Here the input signal is quantized at the sampling rate, with no additional processing.

In [ ]:
# Get the samples
samples = np.multiply(s, sampling_points)
# Implement the hold and quantize the signal
(sh_output, quantized) = hold_quantize(samples = samples)

# Display the sample and hold output, as well as the quantized output
name = 'ADC_quantization_no_dither.pdf'
plot_amplitude(s, sh_output, quantized, name)

The spectrum of the quantized output is now computed using minimum variance spectral estimation.  In the plot, harmonics of the input signal can clearly be observed.  For many applications, this is highly undesirable.

In [ ]:
# For frequency analysis, work at the final sampling rate
downsampled = quantized[downsample_index]

Px = final_spectrum(downsampled = downsampled,  FS = fs)

# Plot the result for figure 2
name = 'ADC_quantization_no_dither_spectrum.pdf'
plot_magnitude(frequency, Px, name)

### Addition of dither
The harmonics observed above are the result of correlation between the quantization error, and the input signal.  In order to avoid this correlation, noise can be added to the input prior to it being quantized.  This is called dithering.  The plot below shows this in the time domain.

In [ ]:
# This time, repeat the process, adding dither noise
dithered_signal = s + ((np.random.rand(duration+1)-0.5)*2*R/(2**b-1))
samples = np.multiply(dithered_signal, sampling_points)

# Implement the hold and quantize the signal
(sh_output, quantized) = hold_quantize(samples = samples)

# Plot figure 3
name = 'ADC_quantization_dither.pdf'
plot_amplitude(s, sh_output, quantized, name)

In frequency domain, no harmonics can be observed.

In [ ]:
# For frequency analysis, work at the final sampling rate
downsampled = quantized[downsample_index]
Px = final_spectrum(downsampled = downsampled, FS = fs)

# Plot the result for figure 4
name = 'ADC_quantization_dither_spectrum.pdf'
plot_magnitude(frequency, Px, name)

## Improving signal to noise ratio
Adding noise to the signal reduces the signal to noise ratio of the converter.  We will now explore techniques for increasing the SNR.

### Oversampling
This is simply the process of sampling the input signal faster than required at the ouptut.  Below the sample and hold, and the quantizer are operating at a rate which is O times faster than before.

In [ ]:
# Now, oversample the data
samples = np.multiply((s+((np.random.rand(duration+1)-0.5)*2*R/(2**b-1))), oversampling_points)

# Implement the hold and quantize the signal
(sh_output, quantized) = hold_quantize(divider = O, samples = samples)

# Plot figure 5
name = 'ADC_quantization_oversampled.pdf'
plot_amplitude(s, sh_output, quantized, name)

The spectrum of the output shows that the signal is now concentrated in the left hand section of the plot and is higher than in the previous plot, while the noise has been spread across the whole plot.  Because the noise power is unchanged, then the noise level in the plot remains the same.

In [ ]:
# For frequency analysis, work at the final sampling rate
downsampled = quantized[os_downsample_index]
Px = final_spectrum(downsampled = downsampled, FS = ofs)

name = 'ADC_quantization_oversampled_spectrum.pdf'
plot_magnitude(os_frequency, Px, name)

### Removing noise
As the noise in the plot is now spread over a larger frequency range, it is possible to filter out some of this noise without affecting the signal, which remains in its original frqeuency band.  By removing noise, then the signal to noise ratio will improve (get larger).  Here we use an FIR filter with a Hann window, which is defined below

In [ ]:
n = np.arange(0, D)
cutoff = 1 / O

h = np.multiply((1-np.cos(2*np.pi*n/(D-1))),
                np.divide(np.sin(np.pi*(n-(D-1)/2)*cutoff),
                          (2*np.pi*(n-(D-1)/2)+np.finfo(float).eps)))
h[int((D+1)/2)-1] = cutoff

The filter is applied to the oversampled input, and the resulting spectrum is plotted below.

In [ ]:
filtered_real = np.convolve(downsampled,h)
Px = final_spectrum(downsampled = filtered_real, FS = ofs)

# Plot the result
name = 'ADC_quantization_oversampled_filtered_spectrum.pdf'
plot_magnitude(os_frequency, Px, name)

### Decimation
The output of the filter, which is still at the high sampling rate, is decimated by the oversampling factor to obtain a result at the desired sampling rate.  Here we plot the signal in the time domain.

In [ ]:
decimation_index = np.arange(0, np.floor(filtered_real.size/O)+1, dtype = int)*O
decimated = filtered_real[decimation_index]

# Kronecker product of two arrays
decimated_for_plotting = np.kron(decimated, np.ones(points_per_sample))

# Allow for FIR lead in
plt.figure(figsize = (12, 6))

plt.plot(tim[499:duration+1],decimated_for_plotting[499:duration+1])

plt.xlim([500*Ts, (500+plot_samples)*Ts])
plt.xlabel('Time (s)', fontsize = 15)
plt.ylim([-R*1.2, R*1.2])
plt.ylabel('Amplitude', fontsize = 15)

if not is_jupyter(): plt.savefig('ADC_quantization_oversampled_decimated.pdf')

The spectrum of the decimated signal shows that the resulting noise level is now lower.  This is because we removed noise in the oversampled signal.

In [ ]:
# compute the final spectrum b
Px = final_spectrum(downsampled = decimated, FS = fs)

# Plot the result
name = 'ADC_quantization_oversampled_decimated_spectrum.pdf'
plot_magnitude(frequency, Px, name)

## Noise shaping
We can further improve the effect of oversampling by also using noise shaping.  Here, the transfer function of the quantization and dither noise is made to be different from that of the input signal.  If we arrange for the quantization and dither noise to pass through a high-pass filter, then we can further reduce the noise in the frequency band that we are interested in (the one with the signal).

Constructing such a system in Python can be quite inefficient, so we will use linear systems theory to directly apply the effect of the filter on the error signal.

In [ ]:
# First calculate the quantization error
ideal_samples = np.multiply(s, oversampling_points)

# Implement the hold
sh_ideal_output = np.convolve(ideal_samples, np.ones(int(points_per_sample/O)))
quantizer_error = sh_ideal_output - quantized


# Then filter it

Ha = np.zeros(int(2*points_per_sample/O)+1)
Ha[0] = 1
Ha[int(points_per_sample/O)] = -2
Ha[int(2*points_per_sample/O)] = 1
Hb = np.ones(1)

filtered_quantizer_error = lfilter(Ha, Hb, quantizer_error)

# And finally add it back to the signal

sh_noise_shaped_output = sh_ideal_output + filtered_quantizer_error

Now produce the quantized signal, and plot it.  Note that we are using the same oversampling factor as above.

In [ ]:
# Quantize the signal
ns_quantized = np.round((2**b-1)*sh_noise_shaped_output/(2*R+np.finfo(float).eps))/(2**b-1)

name = 'ADC_noise_shaped_oversampled.pdf'
plot_amplitude(s, sh_noise_shaped_output, ns_quantized, name)

The spectrum of the signal shows the high-pass nature of the quantization and dither noise.

In [ ]:
# For frequency analysis, work at the current sampling rate
downsampled = sh_noise_shaped_output[os_downsample_index]
# Compute the final spectrum 
Px = final_spectrum(downsampled = downsampled, FS = ofs)

# Plot the result
name = 'ADC_noise_shaped_oversampled_spectrum.pdf'
plot_magnitude(os_frequency, Px, name)

As for the oversampled system, we will apply a filter, which will remove the high frequency noise.  Because of the high-pass nature of the noise, this removes more of the noise than previously where we oversampled, but did not noise shape.

In [ ]:
filtered_real = (2**(b+ns_add_bits)-1)*np.convolve(downsampled*R/(2**b-1),h)/(2*R+np.finfo(float).eps)
filtered_real = np.convolve(downsampled,h)
# Compute the final spectrum 
Px = final_spectrum(downsampled = filtered_real, FS = ofs)

# Plot the result
name = 'ADC_noise_shaped_oversampled_filtered_spectrum.pdf'
plot_magnitude(os_frequency, Px, name)

Finally, we decimated the output of the filter, and obtain our quantized signal, at the desired sampling frequency, with a much higher SNR than from the dithered quantizer at the start of this notebook.

In [ ]:
decimation_index = np.arange(0, np.floor(filtered_real.size)/O, dtype = int) * O
decimated = filtered_real[decimation_index]
decimated_for_plotting = np.kron(decimated, np.ones(points_per_sample))

plt.figure(figsize = (12, 6))
# Allow for FIR lead in
plt.plot(tim[499:duration+1], decimated_for_plotting[499:duration+1])

plt.xlim([500*Ts, (500+plot_samples)*Ts])
plt.xlabel('Time (s)', fontsize = 15)
plt.ylim([-R*1.2, R*1.2])
plt.ylabel('Amplitude', fontsize = 15)

if not is_jupyter(): plt.savefig('ADC_noise_shaped_oversampled_decimated.pdf')

The spectrum shows that the remaining noise is not white due to the noise shaping process.

In [ ]:
# Compute the final spectrum 
Px = final_spectrum(downsampled = decimated, FS = fs)

# Plot the result
name = 'ADC_noise_shaped_oversampled_decimated_spectrum.pdf'
plot_magnitude(frequency, Px, name)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.